In [1]:
import spacy
import pandas as pd
import spacy_transformers

/home/vidur/mediagraph/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vidur/mediagraph/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/vidur/mediagraph/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
nlp = spacy.load('en_core_web_trf')

df = pd.read_csv('/home/vidur/mediagraph/data/TOI_FarmersProtests.csv', delimiter=',', quotechar='"', encoding='utf-8')

/home/vidur/mediagraph/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike

In [5]:
def extract_entities(text):
    if not isinstance(text, str):
        return []
    doc = nlp(text)
    # Filter entities to include only 'PERSON', 'ORG', 'GPE'
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [6]:
text = 'Noida: The rehabilitation package for stalled group housing projects , Yamuna Expressway toll charges and compensation for farmers are among the 30 agendas that will be up for discussion at the YEIDA board meeting on Monday.The board will approve the revised dues of developers of nine defaulter projects stuck over Rs 4,800 crore dues. The revisions will incorporate a two-year zero period due to the Covid pandemic from April 1, 2020, to March 31, 2022.Later, YEIDA will send the developers letters, asking them to clear the dues. At least 10,000 buyers have flats in these projects.The board will also discuss'

In [7]:
extract_entities(text)

[('Noida', 'GPE'),
 ('Yamuna Expressway', 'FAC'),
 ('30', 'CARDINAL'),
 ('YEIDA', 'ORG'),
 ('Monday', 'DATE'),
 ('nine', 'CARDINAL'),
 ('Rs 4,800 crore', 'MONEY'),
 ('two-year', 'DATE'),
 ('April 1, 2020', 'DATE'),
 ('March 31, 2022.Later', 'DATE'),
 ('YEIDA', 'ORG'),
 ('At least 10,000', 'CARDINAL')]

In [19]:
# Replace NaN values with empty strings
df['Text'] = df['Text'].fillna('')

In [20]:
df['Entities'] = df['Text'].apply(extract_entities)

In [21]:
# Convert the entities list to a string
df['Entities'] = df['Entities'].apply(lambda ents: '; '.join([f"{text} ({label})" for text, label in ents]))


In [22]:
# Save the DataFrame back to a CSV file
df.to_csv('output.csv', index=False)


In [8]:
def extract_entities(text):
    if not isinstance(text, str):
        return []
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [9]:
text = "BENGALURU: The state government's decision to provide reservation to contractors belonging to Scheduled Castes (SC) and Scheduled Tribes (ST) in construction contracts has been upheld by the high court. Dismissing a petition filed by HM Vishwanath, a class -1 contractor from Bengaluru, justice B Veerappa has noted that reservation has been provided based on the principles of distributive justice to protect the interests of weaker sections of the society, particularly those belonging to SC-ST category. In a notification issued on July 17, 2017, the state government provided up to 17.15% reservation for contractors belonging to SC category and up to 6.95% reservation for contractors belonging to ST category, with respect to contracts not exceeding Rs 50 lakhs. For this purpose, the Karnataka Transparency in Public Procurement Rules ,2016 was amended to insert rule 27 (A). The rules also provide that only in instances wherein no contractor from these two categories submit a response even after two attempts, a contract relating to such works may be tendered among others"

corrected_entities = extract_entities(text)

In [10]:
for entity in corrected_entities:
    print(entity)

('BENGALURU', 'GPE')
('SC', 'ORG')
('Vishwanath', 'PERSON')
('Bengaluru', 'GPE')
('B Veerappa', 'PERSON')
('July 17, 2017', 'DATE')
('up to 17.15%', 'PERCENT')
('up to 6.95%', 'PERCENT')
('Rs 50 lakhs', 'MONEY')
('the Karnataka Transparency in Public Procurement Rules', 'LAW')
('2016', 'DATE')
('27', 'CARDINAL')
('two', 'CARDINAL')
('two', 'CARDINAL')


In [2]:
import spacy
import pandas as pd
from elasticsearch import Elasticsearch
from difflib import SequenceMatcher
from elasticsearch.helpers import bulk

# Load Spacy model
nlp = spacy.load('en_core_web_trf', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

# Setup Elasticsearch connection
api_key = 'WF9TUmlKTUJ0Q0I2SGtfa25uRTk6bTVGOVJZVzJSdk9DdlR2dzY3Z2Z4Zw=='
es = Elasticsearch(
    "https://my-elasticsearch-project-ee7c3d.es.ap-southeast-1.aws.elastic.cloud:443",
    headers={'Authorization': f'ApiKey {api_key}'}
)

# Setup Elasticsearch index for resolved entities if it doesn't exist
def setup_es_index():
    if not es.indices.exists(index='resolved_entities'):
        es.indices.create(index='resolved_entities', body={
            "mappings": {
                "properties": {
                    "name": {"type": "text"},
                    "aliases": {"type": "text"},
                    "type": {"type": "keyword"}  # e.g., IAS, POL
                }
            }
        })

def index_datasets():
    datasets = [
        {'file_path': '/home/vidur/mediagraph/data/Graph DB data/POL.csv', 'type': 'POL'},
        {'file_path': '/home/vidur/mediagraph/data/Graph DB data/IAS.csv', 'type': 'IAS'},
        {'file_path': '/home/vidur/mediagraph/data/Graph DB data/COMP.csv', 'type': 'COMP'},
        {'file_path': '/home/vidur/mediagraph/data/Graph DB data/DIR.csv', 'type': 'DIR'},
        # Add more datasets as needed
    ]
    actions = []
    for dataset in datasets:
        df = pd.read_csv(dataset['file_path'], delimiter='\t', header=None, names=['id', 'name', 'score'])
        for _, row in df.iterrows():
            action = {
                "_index": "resolved_entities",
                "_source": {
                    'name': row['name'],
                    'aliases': '',
                    'type': dataset['type']
                }
            }
            actions.append(action)
    bulk(es, actions)

# Extract entities using SpaCy
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']]
    return entities

def check_resolved_entities(name):
    body = {
        "query": {
            "multi_match": {
                "query": name,
                "fields": ["name^3", "aliases"],
                "fuzziness": "AUTO"
            }
        },
        "size": 10  # Get top 10 matches
    }
    response = es.search(index="resolved_entities", body=body)
    return response['hits']['hits']

def determine_best_match(name, matches):
    # Sort matches by Elasticsearch's relevance score
    sorted_matches = sorted(matches, key=lambda x: x['_score'], reverse=True)
    return sorted_matches[0]  # Return the top match

def compute_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio() * 100  # Returns percentage

def resolve_entity(entity):
    if not entity:
        return
    name, entity_label = entity
    matches = check_resolved_entities(name)
    if not matches:
        # No match found, entity label remains as is
        print(f"No match found for '{name}'")
        return (name, entity_label)
    else:
        # Process matches to determine best match and merge if necessary
        best_match = determine_best_match(name, matches)
        # Compute similarity score
        match_name = best_match['_source']['name']
        similarity_score = compute_similarity(name.lower(), match_name.lower())
        if similarity_score >= 90:
            # Assign the entity type from the matched entity
            entity_type = best_match['_source']['type']
            print(f"Resolved '{name}' to '{match_name}' with type '{entity_type}' (score: {similarity_score:.2f})")
            return (name, entity_type)
        else:
            # Similarity score is below threshold, entity label remains as is
            print(f"Match for '{name}' is below threshold (score: {similarity_score:.2f})")
            return (name, entity_label)

def main():
    setup_es_index()
    index_datasets()
    # Load data
    df = pd.read_csv('/home/vidur/mediagraph/data/FarmersProtest2024.csv', delimiter=',', quotechar='"', encoding='utf-8')
    df['Text'] = df['Text'].fillna('') 
    df['Entities'] = df['Text'].apply(extract_entities)
    df['Resolved Entities'] = df['Entities'].apply(lambda ents: [resolve_entity(ent) for ent in ents])
    
    # Save the DataFrame back to a CSV file
    df.to_csv('2024output_PreAlias.csv', index=False)

if __name__ == "__main__":
    main()


/home/vidur/mediagraph/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike

Match for 'Haryana' is below threshold (score: 51.85)
Match for 'Haryana' is below threshold (score: 51.85)
Match for 'Jammu & Kashmir Alliance' is below threshold (score: 46.91)
Match for 'PValue' is below threshold (score: 33.33)
Match for 'Noida' is below threshold (score: 29.41)
No match found for 'YEIDA'
No match found for 'YEIDA'
Match for 'Shambu' is below threshold (score: 40.00)
Match for 'US' is below threshold (score: 25.00)
Match for 'UK' is below threshold (score: 33.33)
Match for 'Madhura' is below threshold (score: 85.71)
Match for 'Leicester East' is below threshold (score: 34.29)
Match for 'Claudia Webbe' is below threshold (score: 51.61)
Match for 'USA' is below threshold (score: 85.71)
Match for 'Swaiman Singh' is below threshold (score: 88.00)
Match for 'BKU' is below threshold (score: 33.33)
Match for 'Ekta Ugrahan' is below threshold (score: 66.67)
Match for 'Joginder Singh Ugrahan' is below threshold (score: 77.78)
Match for 'BKU' is below threshold (score: 33.33

In [1]:
resolved_entities

NameError: name 'resolved_entities' is not defined